## Problema a resolver
Crear un estimador k-means con un número de clusters igual al número de géneros.
Entrenar el modelo y indicar el cluster asignado a cada película.

## Entrega del trabajo
Enviar la solución en formato jupyter-notebook antes de las 13:00 pm del Martes 8 de Febrero al correo juan.mondaca@gmail.com 
Grabar el archivo ipynb con su nombre y apellido.

In [58]:
import pandas as pd
import numpy as np

In [2]:
peliculas = pd.read_csv("movies_1.csv")

In [3]:
peliculas.head()

,presupuesto,genero,lenguaje,popularidad,productores,pais,ventas,duracion,titulo,puntuacion,n_votos
0,NaN,Comedy,en,8.387519,Sandollar Productions,United States of America,76578911.0,106.0,Father of the Bride Part II,5.7,173.0
1,NaN,Drama,en,0.894647,Miramax,South Africa,676525.0,106.0,"Cry, the Beloved Country",6.7,13.0
2,3500000.0,Comedy,en,14.569650,New Line Cinema,United States of America,28215918.0,91.0,Friday,7.0,513.0
3,NaN,Comedy,en,8.963037,Paramount Pictures,United States of America,32.0,87.0,Black Sheep,6.0,124.0
4,12000000.0,Comedy,en,9.592265,Universal Pictures,United States of America,41205099.0,92.0,Happy Gilmore,6.5,767.0


In [4]:
peliculas.shape

(1344, 11)

Dividimos los datos en categoricos y numericos

In [7]:
datos_numericos = peliculas.select_dtypes(["int64","float64"]) 
datos_numericos

,presupuesto,popularidad,ventas,duracion,puntuacion,n_votos
0,NaN,8.387519,76578911.0,106.0,5.7,173.0
1,NaN,0.894647,676525.0,106.0,6.7,13.0
2,3500000.0,14.569650,28215918.0,91.0,7.0,513.0
3,NaN,8.963037,32.0,87.0,6.0,124.0
4,12000000.0,9.592265,41205099.0,92.0,6.5,767.0
...,...,...,...,...,...,...
1339,NaN,1.642247,NaN,86.0,3.0,25.0
1340,NaN,0.352816,NaN,11.0,5.5,2.0
1341,NaN,0.110588,NaN,86.0,5.7,3.0
1342,NaN,9.742082,NaN,90.0,5.7,62.0


In [6]:
datos_categoricos = peliculas.select_dtypes([object])
datos_categoricos

,genero,lenguaje,productores,pais,titulo
0,Comedy,en,Sandollar Productions,United States of America,Father of the Bride Part II
1,Drama,en,Miramax,South Africa,"Cry, the Beloved Country"
2,Comedy,en,New Line Cinema,United States of America,Friday
3,Comedy,en,Paramount Pictures,United States of America,Black Sheep
4,Comedy,en,Universal Pictures,United States of America,Happy Gilmore
...,...,...,...,...,...
1339,Horror,en,La Luna Entertainment,United States of America,The Ouija Experiment 2: Theatre of Death
1340,NaN,en,NaN,NaN,Voyage to the Sky
1341,NaN,en,NaN,NaN,The Anatomy of Hate
1342,Horror,en,Campbell Grobman Films,United States of America,Leatherface


#### Datos Categoricos
Trabajaremos los datos categoricos y los codificaremos

In [8]:
#Cantidad de Datos Unicos por columna
numero_valores_unicos ={}
for col in datos_categoricos.columns:
   numero_valores_unicos[col] = datos_categoricos[col].nunique()
numero_valores_unicos


{'genero': 17, 'lenguaje': 29, 'productores': 609, 'pais': 37, 'titulo': 1329}

- Se puede ver que **Productores** y **titulo** no aportan mucha información para el modelo, ya que son demasiados valores únicos, por lo que esas columnas no seran codificadas.

In [9]:
datos_categoricos = datos_categoricos.drop(["productores","titulo"], axis=1)

In [11]:
# Contar los NaN
datos_categoricos.isnull().sum()

genero      51
lenguaje     0
pais        98
dtype: int64

In [ ]:
#Cantidad de Datos Unicos por columna
valores_unicos ={}
for col in datos_categoricos.columns:
   valores_unicos[col] = datos_categoricos[col].value_counts()
valores_unicos

- Tanto **genero** como **pais** tienen datos nulos, por lo que es necesario rellenarlos.
- Para **genero** decidí colocar un genero nuevo como "Otro"
- Para **pais** utilizaré el pais que más se repite que es "United States of America"

In [13]:
datos_categoricos['genero'].fillna('Otro',inplace=True)
datos_categoricos['pais'].fillna('United States of America',inplace=True)
datos_categoricos.isnull().sum()


genero      0
lenguaje    0
pais        0
dtype: int64

In [14]:
# Numero de cluster a realizar, es parte del problema planteado
k = datos_categoricos['genero'].nunique()
k

18

- Como genero sería la columna target, es necesario sacarla para realizar el modelo ya que es un problema de aprendizaje no supervisado.
- Creé un genero nuevo con los NaN por eso no son 17 cluster sino 18.

In [15]:
datos_categoricos_sin_genero = datos_categoricos.drop(["genero"], axis=1)

In [16]:
datos_categoricos_codificados = pd.get_dummies(datos_categoricos_sin_genero)
datos_categoricos_codificados

,lenguaje_bn,lenguaje_cn,lenguaje_da,lenguaje_de,lenguaje_el,lenguaje_en,lenguaje_es,lenguaje_fa,lenguaje_fi,lenguaje_fr,...,pais_South Korea,pais_Spain,pais_Sri Lanka,pais_Sweden,pais_Switzerland,pais_Taiwan,pais_Thailand,pais_Turkey,pais_United Kingdom,pais_United States of America
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1340,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1341,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1342,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Datos Numéricos
Trabajaremos los datos numéricos

In [17]:
# Contar los NaN
datos_numericos.isnull().sum()

presupuesto    588
popularidad      0
ventas         141
duracion         8
puntuacion       0
n_votos          0
dtype: int64

- Se puede apreciar que la cantidad de nulos en la columna **presupuesto** es mucha, 43,75% de los datos, por lo que reemplazar esos valores por el promedio no aportaria información.
- Las otras columnas si seran reemplazadas por el promedio

In [18]:
datos_numericos = datos_numericos.drop(["presupuesto"], axis=1)
for col in datos_numericos.columns:
    datos_numericos[col].fillna(datos_numericos[col].mean(),inplace=True)
datos_numericos.isnull().sum()

popularidad    0
ventas         0
duracion       0
puntuacion     0
n_votos        0
dtype: int64

In [19]:
# Normalizamos los datos, en este caso usaremos StandardScaler
from sklearn.preprocessing import StandardScaler
datos_numericos_normalizados = StandardScaler().fit_transform(datos_numericos)
datos_numericos_normalizados = pd.DataFrame(datos_numericos_normalizados,
                                           columns=datos_numericos.columns)
datos_numericos_normalizados

,popularidad,ventas,duracion,puntuacion,n_votos
0,0.483540,0.672893,0.183377,-0.235106,-0.175398
1,-0.935414,-0.584099,0.183377,0.452410,-0.500797
2,1.654274,-0.128029,-0.426257,0.658665,0.516075
3,0.592528,-0.595302,-0.588827,-0.028851,-0.275052
4,0.711688,0.087080,-0.385615,0.314907,1.032646
...,...,...,...,...,...
1339,-0.793838,0.000000,-0.629469,-2.091399,-0.476392
1340,-1.038023,0.000000,-3.677641,-0.372609,-0.523169
1341,-1.083894,0.000000,-0.629469,-0.235106,-0.521135
1342,0.740059,0.000000,-0.466900,-0.235106,-0.401144


In [22]:
peliculas_procesados = pd.concat([datos_numericos_normalizados,
                                  datos_categoricos_codificados], axis = 1)
peliculas_procesados.shape

(1344, 72)

In [23]:
from sklearn.cluster import KMeans

In [25]:
estimador_kmedias = KMeans(init="k-means++",n_clusters=k,random_state=42)
estimador_kmedias.fit(peliculas_procesados)

KMeans(n_clusters=18, random_state=42)

In [26]:
lclusters = estimador_kmedias.labels_
lclusters

array([ 6, 12,  9, ..., 12,  6, 13])

In [43]:
# Creamos un Datafrane donde agregamos el cluster asignado a cada pelicula, ademas utilizamos 
#los datos numericos y categoricos que sirvieron para construir el modelo, 
#incluimos los titulos de la pelicula.

cluster = pd.DataFrame(lclusters, columns=['Cluster'])
cluster
peliculas_cluster = pd.concat([cluster, peliculas['titulo'],
                               datos_categoricos, datos_numericos, 
                               ], axis = 1)
peliculas_cluster.head()

,Cluster,titulo,genero,lenguaje,pais,popularidad,ventas,duracion,puntuacion,n_votos
0,6,Father of the Bride Part II,Comedy,en,United States of America,8.387519,76578911.0,106.0,5.7,173.0
1,12,"Cry, the Beloved Country",Drama,en,South Africa,0.894647,676525.0,106.0,6.7,13.0
2,9,Friday,Comedy,en,United States of America,14.569650,28215918.0,91.0,7.0,513.0
3,6,Black Sheep,Comedy,en,United States of America,8.963037,32.0,87.0,6.0,124.0
4,9,Happy Gilmore,Comedy,en,United States of America,9.592265,41205099.0,92.0,6.5,767.0


In [78]:
# Calcular la frecuencia de cada genero en los cluster encontrados
resumen = peliculas_cluster.pivot_table(values= 'titulo',index='Cluster',columns= ['genero'],
                                                aggfunc='count')
for col in resumen.columns:
 resumen[col].fillna(0,inplace=True)
resumen.astype(int)

genero,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Horror,Music,Mystery,Otro,Romance,Science Fiction,Thriller,War,Western
Cluster,,,,,,,,,,,,,,,,,,
0,0,0,1,10,0,3,75,0,0,0,0,1,0,0,1,1,1,8
1,4,0,1,21,0,1,13,0,0,2,0,0,4,0,0,0,0,0
2,0,2,1,37,1,43,42,0,0,3,2,1,1,0,1,0,0,2
3,0,0,0,17,0,0,16,0,0,3,0,0,0,1,0,0,0,2
4,1,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,6,0,13,8,0,0,0,0,0,12,1,0,0,0,2
6,6,3,3,106,0,3,32,0,0,23,0,0,0,0,4,10,0,1
7,0,1,1,10,0,0,13,0,0,1,0,0,0,0,0,0,0,0
8,2,2,0,54,1,9,30,2,0,11,0,0,4,0,6,5,0,3


- Podemos apreciar que los cluster no tienen relacion directa con los generos, por tanto no es posible hacer una relación directa.
- Las variables utilizadas no estan relacionadas para generar grupos de comportamiento (cluster) que sean similares a los generos. Tiene otra logica de agrupamiento. 

In [79]:
# La respuesta al problema es el Dataframe peliculas_cluster
peliculas_cluster

,Cluster,titulo,genero,lenguaje,pais,popularidad,ventas,duracion,puntuacion,n_votos
0,6,Father of the Bride Part II,Comedy,en,United States of America,8.387519,7.657891e+07,106.0,5.7,173.0
1,12,"Cry, the Beloved Country",Drama,en,South Africa,0.894647,6.765250e+05,106.0,6.7,13.0
2,9,Friday,Comedy,en,United States of America,14.569650,2.821592e+07,91.0,7.0,513.0
3,6,Black Sheep,Comedy,en,United States of America,8.963037,3.200000e+01,87.0,6.0,124.0
4,9,Happy Gilmore,Comedy,en,United States of America,9.592265,4.120510e+07,92.0,6.5,767.0
...,...,...,...,...,...,...,...,...,...,...
1339,8,The Ouija Experiment 2: Theatre of Death,Horror,en,United States of America,1.642247,3.594685e+07,86.0,3.0,25.0
1340,13,Voyage to the Sky,Otro,en,Otro,0.352816,3.594685e+07,11.0,5.5,2.0
1341,12,The Anatomy of Hate,Otro,en,Otro,0.110588,3.594685e+07,86.0,5.7,3.0
1342,6,Leatherface,Horror,en,United States of America,9.742082,3.594685e+07,90.0,5.7,62.0
